# Task 3
## 1. Env set up
1. ... already set up
---
## 2. Work Sequentially
- using Copilot(Gemini 2.5 pro) to help

#### a. Subtask 1

In [1]:
# Subtask 1
import numpy as np

def linear_layer(x, w, b):
    return np.dot(x, w) + b
    
def relu(x):
    """
    对输入张量 x 执行元素级的 ReLU (Rectified Linear Unit) 操作。
    公式为: f(x) = max(0, x)
    """
    # ===== 在此实现 =====
    return np.maximum(0, x)

def flatten(x):
    """
    将一个四维张量 (N, C, H, W) 展平为一个二维张量 (N, C*H*W)。
    N 是批量大小，需要保持不变。
    """
    # ===== 在此实现 =====
    N = x.shape[0]
    return x.reshape(N, -1)


# comparison experiment
x = np.array([[-2], [-1], [0], [1], [2]])
w1, b1 = np.array([[2]]), np.array([-1])
w2, b2 = np.array([[-1]]), np.array([0.5])

# linear net A
print("Linear Net A:")
out_a1 = linear_layer(x, w1, b1)
out_a2 = linear_layer(out_a1, w2, b2)
print("Output after first linear layer:\n", out_a1.T)
print("Output after second linear layer:\n", out_a2.T)

# net B with Non-linearity
print("\nNet B with Non-linearity:")
out_b1 = linear_layer(x, w1, b1)
out_b2 = relu(out_b1)
out_b3 = linear_layer(out_b2, w2, b2)
print("Output after first linear layer:\n", out_b1.T)
print("Output after ReLU:\n", out_b2.T)
print("Output after second linear layer:\n", out_b3.T)


Linear Net A:
Output after first linear layer:
 [[-5 -3 -1  1  3]]
Output after second linear layer:
 [[ 5.5  3.5  1.5 -0.5 -2.5]]

Net B with Non-linearity:
Output after first linear layer:
 [[-5 -3 -1  1  3]]
Output after ReLU:
 [[0 0 0 1 3]]
Output after second linear layer:
 [[ 0.5  0.5  0.5 -0.5 -2.5]]


##### Analysis
(1)
1. relu in this case changed all negative values to 0 and left positive values unchanged.
2. original input is linearly distributed, and since net A is pure linear transformation, the output is also linearly distributed; while net B with relu breaks this linearity, making the output non-linearly distributed.

(2)
- Non-linear activation funcs are essential because however complex a linear func combination can be, it is still linear. Yet we need the model to learn and handle complex non-linear real-world data, which means non-linearity must be introduced, and this is why non-linear activation funcs matter.

(3)
- The flatten layer is used to convert multi-dimensional data into one-dimensional data, which is much easier to process(as a tensor). This can help to connect different types of layers, like between input layers and dense layers, CNN layers and dense layers, etc. Also we commonly use generators to convert low-dimensional data to high-dimensional data, which is like the opposite of flattening.

#### b. Subtask 2
- Thinking:
1. A fully connected layer needs 100\*100\*1(weights)+1(total bias) = 10001 parameters.
2. a 3\*3 kernel needs 9\*1(weights)+1(bias) = 10 parameters.

In [2]:
import numpy as np

def conv2d(x, w, b, stride=1, padding=0):
    """
    x: input tensor in shape of (N, C_in, H_in, W_in)
    w: weight tensor in shape of (C_out, C_in, H_k, W_k)
    b: bias tensor in shape of (C_out,)
    """
    N, C_in, H_in, W_in = x.shape
    C_out, _, H_k, W_k = w.shape

    H_out = (H_in + 2 * padding - H_k) // stride + 1
    W_out = (W_in + 2 * padding - W_k) // stride + 1

    if padding > 0:
        x_padded = np.pad(x, ((0,0), (0,0), (padding,padding), 
                              (padding,padding)), mode='constant')
    else:
        x_padded = x
        
    out = np.zeros((N, C_out, H_out, W_out))

    for n in range(N):
        for c_out in range(C_out):
            for h_out in range(H_out):
                for w_out in range(W_out):
                    h_start = h_out * stride
                    w_start = w_out * stride
                    window = x_padded[n, :, h_start:h_start+H_k, w_start:w_start+W_k]
                    
                    conv_sum = np.sum(window * w[c_out, :, :, :])
                    out[n, c_out, h_out, w_out] = conv_sum + b[c_out]
    return out

if __name__ == "__main__":
    #阶段一: 验证特征检测
    # 1. 定义一个 5x5 的图像，中心有一个“十字”图案
    image_centered = np.array([[
        [0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 1, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0]
    ]], dtype=np.float32).reshape(1, 1, 5, 5)

    # 2. 设计一个 3x3 的卷积核，当它滑过输入图像时，如果它对应的图像区域与十字图案完全一致，它计算出的结果应该是最大的。
    kernel_cross = np.array([[
        [0, 1, 0],
        [1, 1, 1],
        [0, 1, 0]
    ]], dtype=np.float32).reshape(1, 1, 3, 3)
    bias_zero = np.array([0], dtype=np.float32)
    
    # 3. 执行卷积，观察输出
    output_centered = conv2d(image_centered, kernel_cross, bias_zero, stride=1, padding=0)
    print("Output for centered cross pattern:\n", output_centered[0, 0])


    # 阶段二: 平移不变性
    # 1. 创建一个新图像，将“十字”图案向右下方平移一格
    image_centered = np.array([[
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 1, 1],
        [0, 0, 0, 1, 0]
    ]], dtype=np.float32).reshape(1, 1, 5, 5)
    # 2. 使用完全相同的卷积核进行卷积，并观察输出
    
    output_shifted = conv2d(image_centered, kernel_cross, bias_zero, stride=1, padding=0)
    print("Output for shifted cross pattern:\n", output_shifted[0, 0])

    

Output for centered cross pattern:
 [[2. 2. 2.]
 [2. 5. 2.]
 [2. 2. 2.]]
Output for shifted cross pattern:
 [[0. 0. 1.]
 [0. 2. 2.]
 [1. 2. 5.]]


##### Analysis
1. Locality: The convolution operation yields a feature map where each output value is influenced by a small, localized region of the input image. This is because the convolution kernel covers only a small part of the input at each iteration and computes a weighted sum and add a bias for that region. It has nothing to do with their absolute positions and other parts of the image.
2. Translation Invariance: The convolution operation is translation invariant, because a translation of the feature(the cross's moving) in the input image will result in a corresponding translation of the feature(value '5.') in the output feature map, which is shown as the same value representing the same feature in different positions. It's just the same operation in another position.
3. This means CNN can identify features effectively.

#### c. Subtask 3

In [3]:
def max_pool2d(x, kernel_size=2, stride=2):
    """
    x: input tensor of shape (N, C, H, W)
    """
    N, C, H_in, W_in = x.shape
    H_out = (H_in - kernel_size) // stride + 1
    W_out = (W_in - kernel_size) // stride + 1

    out = np.zeros((N, C, H_out, W_out))

    for n in range(N):
        for c in range(C):
            for h_out in range(H_out):
                for w_out in range(W_out):
                    h_start = h_out * stride
                    w_start = w_out * stride
                    window = x[n, c, h_start:h_start+kernel_size, w_start:w_start+kernel_size]
                    
                    out[n, c, h_out, w_out] = np.max(window)
    return out

print("Max Pooling Example:")
# feature map with a simple pattern
feature_map_1 = np.array([[
    [1, 2, 2, 0],
    [4, 5, 3, 1],
    [2, 3, 9, 9],
    [0, 2, 9, 9]
]], dtype=np.float32).reshape(1, 1, 4, 4)

feature_map_2 = np.array([[
   [3, 2, 1, 4],
   [2, 1, 3, 1],
   [3, 9, 9, 1],
   [1, 9, 9, 2],
]], dtype=np.float32).reshape(1, 1, 4, 4)

pooled_1 = max_pool2d(feature_map_1, kernel_size=2, stride=2)
pooled_2 = max_pool2d(feature_map_2, kernel_size=2, stride=2)

print("Original feature map 1:\n", feature_map_1[0, 0])
print("Original feature map 2:\n", feature_map_2[0, 0])

print("Pooled feature map 1:\n", pooled_1[0, 0])
print("Pooled feature map 2:\n", pooled_2[0, 0])

Max Pooling Example:
Original feature map 1:
 [[1. 2. 2. 0.]
 [4. 5. 3. 1.]
 [2. 3. 9. 9.]
 [0. 2. 9. 9.]]
Original feature map 2:
 [[3. 2. 1. 4.]
 [2. 1. 3. 1.]
 [3. 9. 9. 1.]
 [1. 9. 9. 2.]]
Pooled feature map 1:
 [[5. 3.]
 [3. 9.]]
Pooled feature map 2:
 [[3. 4.]
 [9. 9.]]


##### Analysis
- (1) we can see that the output feature map retains the most prominent features('9.') from the input feature map. The strongest feature is always retained after pooling, which means translation invariance is achieved to some extent: change the position of the feature won't change the feature value, so it still gets detected and retained.
- (2) for this set up, the output of the pooling layer is halved in both height and width, effectively reducing the size of the feature map while retaining the most salient features. With a stride of s, the output dimensions will be reduced by a factor of s in both height and width, resulting a W*H/s^2 output feature map.
- (3):
    1. if adopting "Average Pooling", the output feature map will have averaged values in each pooling region, which may smooth out the features and make them less distinct. This could lead to loss of important details, especially if the features are not uniformly distributed.
    2. when regarding feature selection, max pooling tends to effectively highlight the most prominent features, while average pooling may dilute them by averaging with less significant values. Just as their names suggest, "max" and "average".

#### d. Subtask 4
##### Answer:
- (1):
    1. every element in the output probability vector is between 0 and 1 and they sum up to 1.0 .
    2. These features makes softmax ideal for multi-class classification because its output can be interpreted as a probability distribution, which can be further interpreted as the model's confidence in each class.


In [4]:
import numpy as np
import gzip
import os
import struct
from array import array

# (在此之前应有已实现的 conv2d, relu, max_pool2d, flatten,linear_layer函数)
# 固定随机种子，保证权重初始化一致
np.random.seed(114514)

def softmax(logits):
    """
    实现Softmax函数。
    logits: input tensor of shape (N, num_classes)
    """
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

# --- MNIST 数据集读取函数 ---
def read_images(filename):
    """
    读取MNIST图像文件
    参数:
      filename: MNIST图像文件路径
    返回:
      images: 图像数组列表
    """
    with open(filename, 'rb') as file:
        magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
        if magic != 2051:
            raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
        
        image_data = array("B", file.read())
        
    images = []
    for i in range(size):
        img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
        img = img.reshape(rows, cols)
        images.append(img)
    
    return images

# =========================================================
# ===== 任务：请根据下面的规约，在此处实现 TinyCNN_for_MNIST 类 =====
# =========================================================
#
# --- 模型架构规约 ---
# 1. 构造函数 `__init__(self)`:
# 架构固定：Conv(1->4, k=3, stride=1, pad=1) -> ReLU -> MaxPool(2x2, s=2) -> Flatten -> Linear(->10 类)
# 2. 前向传播方法 `forward(self, x)`:
#    - 接收一个形状为 (N, 1, 28, 28) 的张量 x。
#    - 按照以下顺序依次调用你实现的算子：
#      Conv2d -> ReLU -> MaxPool2d -> Flatten -> Linear -> Softmax
#    - 返回最终的 logits (Linear层输出) 和 probs (Softmax层输出)。

class TinyCNN_for_MNIST:
    def __init__(self):
        
        self.conv_w = np.random.randn(4, 1, 3, 3) * 0.01
        self.conv_b = np.zeros(4)

        self.lr_w = np.random.randn(784, 10) * 0.01
        self.lr_b = np.zeros(10)

    def forward(self, x):
        x = conv2d(x, self.conv_w, self.conv_b,padding=1)
        x = relu(x)
        x = max_pool2d(x, 2, 2)
        x = flatten(x)
        logits = linear_layer(x, self.lr_w, self.lr_b)
        probs = softmax(logits)
        return logits, probs


# --- 测试脚本 ---
if __name__ == "__main__":
    # 1. 设置 MNIST 测试集文件路径
    # !! 请将此路径修改为你自己的文件路径
    mnist_test_file = '../mnist/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte'

    if not os.path.exists(mnist_test_file):
        print(f"错误：找不到 MNIST 测试集文件 '{mnist_test_file}'")
    else:
        # 2. 加载所有测试图像
        test_images = read_images(mnist_test_file)
        # 3. 选取第一张图像作为测试输入
        first_test_image = test_images[0]
        # 4. 预处理图像
        input_tensor = (first_test_image.astype(np.float32) / 255.0 - 0.5) * 2.0
        input_tensor = np.expand_dims(input_tensor, axis=(0, 1))
        # 5. 实例化模型并执行前向传播
        model = TinyCNN_for_MNIST()
        logits, probs = model.forward(input_tensor)

        print("Input Tensor Shape:", input_tensor.shape)
        print("Logits shape:", logits.shape, "Probs shape:", probs.shape)
        np.set_printoptions(precision=8, suppress=False)
        print("\nLogits:", logits[0])
        print("Probs:", probs[0])
        print("\nChecksum logits sum:", float(np.sum(logits)))
        print("Checksum probs sum:", float(np.sum(probs)))

Input Tensor Shape: (1, 1, 28, 28)
Logits shape: (1, 10) Probs shape: (1, 10)

Logits: [ 0.00223343 -0.00416911  0.00250573  0.00267944  0.00646766  0.0015071
 -0.00022611  0.0042225   0.00018825  0.01054704]
Probs: [0.09996308 0.09932511 0.09999031 0.10000768 0.10038725 0.0998905
 0.09971752 0.10016211 0.09975885 0.1007976 ]

Checksum logits sum: 0.025955935726856137
Checksum probs sum: 1.0


##### Assessment:
1. each prob value is close to 0.1, which is expected for a un-trained model
2. they sum up to 1.0, which means softmax is working correctly
3. the logits are small and close to 0.

#### e. Subtask 5

In [5]:
# Subtask 5
import torch
import torch.nn as nn

class PyTorch_CNN(nn.Module):
    def __init__(self):
        super(PyTorch_CNN, self).__init__()
        self.conv = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(4 * 14 * 14, 10)  # 4 channels, 14x14 after pooling

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        logits = self.fc(x)
        probs = torch.softmax(logits, dim=1)
        return logits, probs
    
pytorch_model = PyTorch_CNN()
numpy_model = TinyCNN_for_MNIST()

# weight transfer
with torch.no_grad():
    conv_w_torch =  torch.from_numpy(numpy_model.conv_w).float()
    conv_b_torch =  torch.from_numpy(numpy_model.conv_b).float()
    fc_w_torch =  torch.from_numpy(numpy_model.lr_w.T).float()
    fc_b_torch =  torch.from_numpy(numpy_model.lr_b).float()

    pytorch_model.conv.weight.data = conv_w_torch
    pytorch_model.conv.bias.data = conv_b_torch
    pytorch_model.fc.weight.data = fc_w_torch
    pytorch_model.fc.bias.data = fc_b_torch
    
mnist_test_file = '../mnist/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte'
if not os.path.exists(mnist_test_file):
    print(f"Error: '{mnist_test_file}' not found")
else:
    test_images = read_images(mnist_test_file)
    first_test_image = test_images[0]
    
    numpy_input = (first_test_image.astype(np.float32) / 255.0 - 0.5) * 2.0
    numpy_input = np.expand_dims(numpy_input, axis=(0, 1))
    
    torch_input = torch.from_numpy(numpy_input)
    
    numpy_logits, numpy_probs = numpy_model.forward(numpy_input)
    torch_logits, torch_probs = pytorch_model.forward(torch_input)
    
    print("Numpy Logits:", numpy_logits)
    print("Torch Logits:", torch_logits.detach().numpy())
    
    print("\nNumpy Probs:", numpy_probs)
    print("Torch Probs:", torch_probs.detach().numpy())

    are_close = np.allclose(numpy_logits, torch_logits.detach().numpy(), atol=1e-6)
    print(f"\nAre the outputs of the two models numerically close? {are_close}")

Numpy Logits: [[ 0.00353601 -0.00597512 -0.01245221  0.00474773 -0.00172211  0.00710951
   0.00555682 -0.00088172  0.00028607 -0.00289982]]
Torch Logits: [[ 0.00353601 -0.00597512 -0.01245222  0.00474773 -0.00172211  0.00710952
   0.00555682 -0.00088171  0.00028607 -0.00289982]]

Numpy Probs: [[0.10037968 0.09942948 0.09878755 0.10050138 0.09985325 0.10073903
  0.10058273 0.0999372  0.10005398 0.09973572]]
Torch Probs: [[0.10037968 0.09942947 0.09878755 0.10050138 0.09985325 0.10073902
  0.10058273 0.09993721 0.10005397 0.09973572]]

Are the outputs of the two models numerically close? True


#### --- 4. 完成分析报告(mostly learned from AI) ---

1. 功能性对比 ：除了我们实现的版本，PyTorch 的 nn.Conv2d 还提供了哪些我们的 NumPy 版本不具备的进阶功能或参数？
    - groups: This parameter allows for grouped convolutions, where input channels are split into groups and each group is convolved with its own set of filters. When groups equals in_channels, it becomes a depthwise convolution.
    - dilation: This enables dilated or "atrous" convolution, which increases the receptive field of the kernel without increasing the number of parameters by inserting gaps between kernel elements.
    - Automatic Backend Optimization: It can use highly optimized libraries like cuDNN (on GPU) or MKL (on CPU) under the hood.
    - Data Type Handling: It seamlessly handles various data types (e.g., float16, float32, bfloat16).

2. 性能对比：为何 PyTorch 的运算速度会比我们手写的 Python for 循环快几个数量级？请从底层实现语言、并行计算策略、以及可能的算法优化等角度进行分析。
    - **Underlying Language**: Our implementation uses nested Python for loops while PyTorch's core operations are implemented in highly optimized, compiled C++ and CUDA code, which executes directly on the hardware and is much faster.
    - **Parallel Computation**: Our for loop implementation is inherently sequential. PyTorch is designed for parallelism. Parallel computation yields significant speedups, especially on GPUs.
    - **Optimized Algorithms**: For convolutions, our naive implementation has a high time complexity. PyTorch (via libraries like cuDNN) can use much faster algorithms like Winograd or FFT-based convolution, which are mathematically equivalent but computationally far more efficient, especially for certain kernel sizes.

3. 核心优势对比 ：在你看来，使用 PyTorch 这类框架相对于从零手写，其最根本、最无法替代的优势是什么？
    The most fundamental and irreplaceable advantage is **Automatic Differentiation (Autograd)**.
    - While we have implemented the forward pass, we haven't touched the backward pass (backpropagation), which is necessary for training. Manually deriving and implementing the gradients for every layer is incredibly complex, tedious, and error-prone. PyTorch's autograd engine automatically tracks all operations on tensors. When we call .backward() on a final loss value, it automatically computes the gradients of that loss with respect to all model parameters. This single feature abstracts away the most difficult part of building and training neural networks, allowing developers to focus on model architecture and experimentation, which dramatically accelerates research and development.